In [1]:
import cv2
import csv
import time
import os
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import *
import tkinter.messagebox
import mediapipe as mp
import numpy as np
import mtcnn
from architecture import *
from train_v2 import normalize, l2_normalizer
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model
from PIL import Image
from PIL import ImageTk
import pickle

end_flag = 0
user_flag = 0

#now = datetime.now()
#now_time = now.hour * 60 + now.minute
#now_time_str = str(now.hour) + ":" + str(now.minute)

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

dist_check = []
final_counter = []  #所有動作函式的引入變數

counter = 0  #二頭
stage = None

lateral_raise_counter = 0  #側平舉
lateral_raise_stage = None

shoulder_press_counter = 0  #肩舉
shoulder_press_stage = None

squat_counter = 0
squat_stage = None

name = []
final_name = []
rec_name = 'unknown'
str_name = []


final_Biceps_counter = []
final_lateral_raise_counter = []
final_shoulder_press_counter = []
final_squat_counter = []

confidence_t = 0.99
recognition_t = 0.4
required_size = (160, 160)


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
動作辨識

In [ ]:
def mediapipe(image, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


def Biceps(image, final_counter):  #二頭彎舉(左)
    global counter
    global stage
    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        l_shoulder = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
        ]
        l_elbow = [
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
        ]
        l_wrist = [
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
        ]
        l_hip = [
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        ]
        r_shoulder = [
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
        ]
        r_elbow = [
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
        ]
        r_wrist = [
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
        ]
        r_hip = [
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
        ]
        # Calculate angle
        l_angle_elbow = calculate_angle(l_shoulder, l_elbow, l_wrist)
        l_angle_shoulder = calculate_angle(l_hip, l_shoulder, l_elbow)
        r_angle_elbow = calculate_angle(r_shoulder, r_elbow, r_wrist)
        r_angle_shoulder = calculate_angle(r_hip, r_shoulder, r_elbow)
        # 顯示關節 angle
       # cv2.putText(image, str(angle_elbow),
                   # tuple(np.multiply(elbow, [640, 480]).astype(int)),
                  #  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,
                   # cv2.LINE_AA)
       # cv2.putText(image, str(angle_shoulder),
                 #   tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                 #   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,
                  #  cv2.LINE_AA)
        # Curl counter logic
        #if cv2.waitKey(1) & 0xFF == ord('d') and counter!=0:
        #counter -=1
        if l_angle_elbow > 160 and l_angle_shoulder < 20 and r_angle_elbow > 160 and r_angle_shoulder < 20:
            stage = "down"
        if l_angle_elbow < 20 and stage == 'down' and l_angle_shoulder < 20 and r_angle_elbow < 20 and r_angle_shoulder < 20:
            stage = "up"
            counter += 1
            #print(counter)

    except:
        pass

    # Render curl counter
    # Setup status box
    #cv2.rectangle(image, (0,0), (200,78), (245,117,16), -1)

    # Rep data
    cv2.putText(image, 'Biceps Curl', (12, 27), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter), (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                (255, 255, 255), 2, cv2.LINE_AA)

    # Stage data
    #cv2.putText(image, 'STAGE', (65,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    #cv2.putText(image, stage, (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    #final_counter = counter
    return counter


def lateral_raise(image, final_counter):  #側平舉(左)
    global lateral_raise_counter
    global lateral_raise_stage
    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        shoulder = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
        ]
        elbow = [
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
        ]
        wrist = [
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
        ]
        hip = [
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        ]
        # Calculate angle
        angle_elbow = calculate_angle(shoulder, elbow, wrist)
        angle_shoulder = calculate_angle(hip, shoulder, elbow)
        l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x
        l_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
        hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
        ans = l_elbow - l_shoulder  #手肘-肩膀 X
        ans1 = wrist_y - hip_y  #手肘-臀部 Y

        # 顯示關節 angle
        cv2.putText(image, str(angle_elbow),
                    tuple(np.multiply(elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(image, str(angle_shoulder),
                    tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,
                    cv2.LINE_AA)
        #cv2.putText(image, str(l_shoulder), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, str(l_elbow), tuple(np.multiply(shoulder, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        # Curl counter logic
        #if cv2.waitKey(1) & 0xFF == ord('d') and counter!=0:
        #counter -=1
        if angle_elbow > 150 and angle_shoulder < 20 and ans1 < 0.4:
            lateral_raise_stage = "down"
        if angle_elbow > 150 and lateral_raise_stage == 'down' and angle_shoulder > 80 and ans > 0.151:
            lateral_raise_stage = "up"
            lateral_raise_counter += 1
            #print(lateral_raise_counter)
            #print('ans: ' + str(ans))
            #print('angle_elbow: '+ str(l_elbow))
        # print('angle_shoulder: '+ str(l_shoulder))

    except:
        pass

    # Rep data
    cv2.putText(image, 'lateral_raise', (12, 107), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(lateral_raise_counter), (75, 155),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)

    return lateral_raise_counter


def shoulder_press(image, final_counter):  #肩舉
    global shoulder_press_counter
    global shoulder_press_stage

    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        shoulder = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
        ]
        elbow = [
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
        ]
        wrist = [
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
        ]
        hip = [
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        ]
        # Calculate angle
        angle_elbow = calculate_angle(shoulder, elbow, wrist)
        angle_shoulder = calculate_angle(hip, shoulder, elbow)
        l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x
        l_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x

        ans = l_elbow - l_shoulder  #手肘-肩膀 X
        x = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
        y = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y  #0.56/0.25
        # 顯示關節 angle
        #cv2.putText(image, str(angle_elbow), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, str(angle_shoulder), tuple(np.multiply(shoulder, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, str(y), tuple(np.multiply(elbow, [640, 480]).astype(int)) ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        if angle_elbow < 50 and angle_shoulder < 57 and angle_shoulder > 20:
            #x=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
            #print('down: '+str(x))
            shoulder_press_stage = "down"
        if angle_elbow > 140 and shoulder_press_stage == 'down' and angle_shoulder > 140:
            shoulder_press_stage = "up"
            #x=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
            #print('up :'+str(x))
            shoulder_press_counter += 1
            #print(shoulder_press_counter)
            #print('ans: ' + str(ans))
            #print('angle_elbow: '+ str(l_elbow))
        # print('angle_shoulder: '+ str(l_shoulder))

    except:
        pass

    # Rep data
    cv2.putText(image, 'shoulder_press', (12, 187), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(shoulder_press_counter), (75, 237),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)

    return shoulder_press_counter


def squat(image, final_counter):  #深蹲

    global squat_counter
    global squat_stage

    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        r_shoulder = [
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
        ]
        l_shoulder = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
        ]
        r_hip = [
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
        ]
        l_hip = [
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        ]
        r_knee = [
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
        ]
        l_knee = [
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
        ]
        r_ankle = [
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
        ]
        l_ankle = [
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
        ]
        # Calculate angle
        angle_r_knee = calculate_angle(r_hip, r_knee, r_ankle)
        angle_l_knee = calculate_angle(l_hip, l_knee, l_ankle)
        angle_r_hip = calculate_angle(r_shoulder, r_hip, r_knee)
        angle_l_hip = calculate_angle(l_shoulder, l_hip, l_knee)
        l_knee_x = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x
        r_knee_x = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x
        l_knee_value = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
        r_knee_value = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
        l_ankle_value = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y

        ans_l = l_ankle_value - l_knee_value  #腳踝-膝蓋 Y

        if l_knee_value >= r_knee_value:
            ans_knee = l_knee_value - r_knee_value
        else:
            ans_knee = r_knee_value - l_knee_value
        #x=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
        #y=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
        # 顯示關節 angle
        #cv2.putText(image, str(angle_r_knee), tuple(np.multiply(r_knee, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, str(angle_l_knee), tuple(np.multiply(l_knee, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, str(y), tuple(np.multiply(elbow, [640, 480]).astype(int)) ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        #print('r_knee: '+ str(angle_r_knee))
        #print('l_knee: '+ str(angle_l_knee))
        if angle_l_knee > 160 and ans_l > 0 and angle_r_knee > 160:
            #x=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
            #print('down: '+str(x))
            squat_stage = "up"
        if ans_knee < 0.05 and angle_r_hip > 80 and angle_l_hip > 80 and angle_r_hip < 140 and angle_l_hip < 140 and angle_l_knee < 125 and angle_r_knee < 125 and squat_stage == 'up' and ans_l < 0.158 and ans_l > 0 and angle_l_knee > 70 and angle_r_knee > 70:
            squat_stage = "down"
            #x=landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x
            #print('up :'+str(x))
            squat_counter += 1
            #print('差距: '+str(ans_knee))
            #print('l_hip: '+ str(angle_l_hip))
        # print('r_hip: '+ str(angle_r_hip))
        #print('l_knee: '+ str(angle_l_knee))
        #print('r_knee: '+ str(angle_r_knee))
        #print('ans_knee '+ str(ans_knee))

        #print(shoulder_press_counter)
        #print('ans: ' + str(ans))
        #print('value: '+ str(ans_l))
        #print('l: '+ str(angle_l_knee))
        #print('r: '+ str(angle_r_knee))
        # print('angle_shoulder: '+ str(l_shoulder))

    except:
        pass

    # Rep data
    cv2.putText(image, 'squat', (12, 267), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(squat_counter), (75, 317), cv2.FONT_HERSHEY_SIMPLEX,
                1.5, (255, 255, 255), 2, cv2.LINE_AA)
    #cv2.putText(image, 'l_squat', (12,267), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
    #cv2.putText(image, str(l_squat_counter), (75,317), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

    return squat_counter

In [ ]:
臉部

In [ ]:
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)


def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode


def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict


def draw_landmark(image, results):
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66),
                               thickness=2,
                               circle_radius=2),
        mp_drawing.DrawingSpec(color=(245, 66, 230),
                               thickness=2,
                               circle_radius=2))


def detect(img, detector, encoder, encoding_dict):

    #global required_shape
    #global face_encoder
    #global path_m
    #global face_encoder
    #global encodings_path
    #global face_detector
    #global encoding_dict

    global name
    global dist_check

    final_name = 'unknown'
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    for res in results:
        if res['confidence'] < confidence_t:
            continue
        face, pt_1, pt_2 = get_face(img_rgb, res['box'])
        encode = get_encode(encoder, face, required_size)
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        name = 'unknown'
        distance = float("inf")
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            #dist=dist_check
            #print('unknown: '+str(dist_check))
            if dist < recognition_t and dist < distance:
                name = db_name
                #print('know: '+str(dist_check))
                distance = dist

        if name == 'unknown':
            cv2.rectangle(img, pt_1, pt_2, (0, 0, 255), 2)
            cv2.putText(img, name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 255), 1)
        else:
            final_name = name
            cv2.rectangle(img, pt_1, pt_2, (0, 255, 0), 2)
            cv2.putText(img, name + f'__{distance:.2f}',
                        (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)
    return img, final_name

In [ ]:
個人

In [ ]:
def user_start():  #個人介面 "開始"
    global now_day_str
    global user_flag
    global flag
    global frame
    global pose
    global imgg
    global img
    global rec_name
    global final_name
    global win
    global cap
    global results
    global window

    global final_Biceps_counter
    global final_lateral_raise_counter
    global final_shoulder_press_counter
    global final_squat_counter

    global counter
    global lateral_raise_counter
    global shoulder_press_counter
    global squat_counter
    
    now = datetime.now()
    now_time = now.hour * 60 + now.minute
    
    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()

            if not ret:
                print("CAM NOT OPEND")
                break

            img = frame

            frame, results = mediapipe(img, pose)

            draw_landmark(frame, results)
            #print(11)
            final_Biceps_counter = Biceps(frame, final_Biceps_counter)  #二頭彎舉次數
            #print(22)
            final_lateral_raise_counter = lateral_raise(
                frame, final_lateral_raise_counter)  #側平舉次數
            final_shoulder_press_counter = shoulder_press(
                frame, final_shoulder_press_counter)  #肩舉
            final_squat_counter = squat(frame, final_squat_counter)

            str_Biceps_counter = str(final_Biceps_counter)  #二頭彎舉次數 轉成 string
            str_lateral_raise_counter = str(final_lateral_raise_counter)
            str_shoulder_press_counter = str(final_shoulder_press_counter)
            str_squat_counter = str(final_squat_counter)
            #draw_landmark(frame ,results)

            cv2.imshow('camera', frame)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                counter=0
                lateral_raise_counter=0
                shoulder_press_counter=0
                squat_counter=0
                now1 = datetime.now()
                end_time = now1.hour * 60 + now1.minute
                break
            if key & 0xFF == ord('b') and counter != 0:
                counter -= 1
            if key & 0xFF == ord('l') and lateral_raise_counter != 0:
                lateral_raise_counter -= 1
            if key & 0xFF == ord('p') and shoulder_press_counter != 0:
                shoulder_press_counter -= 1
            if key & 0xFF == ord('s') and squat_counter != 0:
                squat_counter -= 1

    #now1 = datetime.now()
    #end_time = now1.hour * 60 + now1.minute
    spend_time = end_time - now_time
    now_day_str = str(now.year)+"/"+ str(now.month) + "/" + str(now.day)
    #if end_flag == 0:
    with open(rec_name + '.csv', 'a+', newline = '' ,encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile)  #以空白分隔欄位，建立 CSV 檔寫入器
        csv_name = rec_name + '.csv'
        if not os.path.getsize(csv_name):
            writer.writerow(['姓名', '二頭', '側平舉', '肩舉', '深蹲','日期', '分鐘'])
        writer.writerow([rec_name, str(final_Biceps_counter), str(final_lateral_raise_counter),str(final_shoulder_press_counter), str(final_squat_counter) ,now_day_str, spend_time])

    cap.release()
    cv2.destroyAllWindows()


def exit():
    global user_flag
    global user
    user.destroy()
    user_flag = 0

def record():
    global rec_name
    #print(1)
    root = tk.Tk()
    root.title('健身紀錄')
    root.geometry("720x550+650+200")

    TableMargin = Frame(root, width=500)
    TableMargin.pack(side=TOP)

    scrollbarx = Scrollbar(TableMargin, orient=HORIZONTAL)
    scrollbary = Scrollbar(TableMargin, orient=VERTICAL)

    tree = ttk.Treeview(TableMargin,
                        columns=("姓名", "二頭", "側平舉", '肩舉', '深蹲' ,'分鐘', '日期'),
                        height=400,
                        selectmode="extended",
                        yscrollcommand=scrollbary.set,
                        xscrollcommand=scrollbarx.set)
    scrollbary.config(command=tree.yview)
    scrollbary.pack(side=RIGHT, fill=Y)
    scrollbarx.config(command=tree.xview)
    scrollbarx.pack(side=BOTTOM, fill=X)
    tree.heading('姓名', text="姓名", anchor=W)
    tree.heading('二頭', text="二頭", anchor=W)
    tree.heading('側平舉', text="側平舉", anchor=W)
    tree.heading('肩舉', text="肩舉", anchor=W)
    tree.heading('深蹲', text="深蹲", anchor=W)
    tree.heading('日期', text="日期", anchor=W)
    tree.heading('分鐘', text="分鐘", anchor=W)
    tree.column('#0', stretch=NO, minwidth=0, width=0)
    tree.column('#1', stretch=NO, minwidth=0, width=120)
    tree.column('#2', stretch=NO, minwidth=0, width=120)
    tree.column('#3', stretch=NO, minwidth=0, width=120)
    tree.column('#4', stretch=NO, minwidth=0, width=120)
    tree.column('#5', stretch=NO, minwidth=0, width=120)
    tree.column('#6', stretch=NO, minwidth=0, width=120)
    tree.column('#7', stretch=NO, minwidth=0, width=120)
    tree.pack()

    with open(rec_name + '.csv', newline='', encoding='utf-8-sig') as csvfile:

        csv_reader = csv.DictReader(csvfile, delimiter=',')
        for row in csv_reader:
            one = row['姓名']
            two = row['二頭']
            three = row['側平舉']
            four = row['肩舉']
            five = row['深蹲']
            six = row['分鐘']
            seven = row['日期']
            tree.insert("", 0, values=(one, two, three, four, five, six, seven))

    root.mainloop()


In [ ]:
初始

In [ ]:
def sign_in():
    global user
    global user_flag
    global flag
    global frame
    global pose
    global imgg
    global img
    global rec_name
    global final_name
    global win
    global cap
    global results
    global window

    global final_Biceps_counter
    global final_lateral_raise_counter
    global final_shoulder_press_counter
    global final_squat_counter

    global counter
    global lateral_raise_counter
    global shoulder_press_counter
    global squat_counter

    required_shape = (160, 160)
    face_encoder = InceptionResNetV2()
    path_m = "facenet_keras_weights.h5"
    face_encoder.load_weights(path_m)
    encodings_path = 'encodings/encodings.pkl'
    face_detector = mtcnn.MTCNN()
    encoding_dict = load_pickle(encodings_path)

    flag = 0
    user_flag = 1

    cap = cv2.VideoCapture(0)

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            if end_flag == 1:
                break

            ret, frame = cap.read()

            if not ret:
                print("CAM NOT OPEND")
                break

            #img = frame

            #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            #if flag == 0:
            frame, rec_name = detect(frame, face_detector, face_encoder,
                                     encoding_dict)  #登入後不再檢測人臉  留?

            #frame ,results = mediapipe(img ,pose)

            if rec_name != 'unknown':  #and flag==0:           #辨識到已知人臉
                #flag = 1
                tkinter.messagebox.showinfo(title='Hello',
                                            message='歡迎:' + rec_name)
                cap.release()
                cv2.destroyAllWindows()
                window.destroy()

                user = tk.Tk()
                user.title('user')
                user.geometry("720x550+650+200")
                user.resizable(width=0, height=0)
                img_user = ImageTk.PhotoImage(Image.open("user.jpg"))
                label_user = tk.Label(user,
                                      text=' 個 人 介 面 ',
                                      justify=tk.LEFT,
                                      image=img_user,
                                      compound=tk.CENTER,
                                      font=('華文行楷', 30),
                                      fg="black")
                label_user.pack()
                button_user = tk.Button(user,
                                        text="開始",
                                        height=2,
                                        width=15,
                                        bd=5,
                                        command=user_start)
                button_user.place(x=120, y=400)
                button_user1 = tk.Button(user,
                                         text="紀錄",
                                         height=2,
                                         width=15,
                                         bd=5,
                                         command=record)
                button_user1.place(x=310, y=400)
                button_user2 = tk.Button(user,
                                         text="登出",
                                         height=2,
                                         width=15,
                                         bd=5,
                                         command=exit)
                button_user2.place(x=500, y=400)

                user.mainloop()
            
            cv2.putText(frame, 'if no account press q', (12, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.imshow('camera', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


def hello():
    global final_name
    global folder
    global result
    global button_pic
    global button_pic1
    global button_pic2
    
    var = entry.get()
    final_name = var
    folder = os.path.exists('C:\\test_face\\Face-recognition\\Faces\\' +
                            final_name)
    if folder:  #檔案存在
        #tkinter.messagebox.showinfo(title='Hello', message='名字重複，請重新輸入')
        str_name = '# 姓名重複，請重新輸入!'
        button_pic1['state'] = tk.DISABLED
        #result.config(fg="red")
        result.set("\n\n\n\t\t\t          "+str_name)
    else:
        #tkinter.messagebox.showinfo(title='Hello', message='成功! 請按拍照按鈕')
        str_name = '# 姓名可用!'
        button_pic1['state'] = tk.NORMAL
        #result.config(fg="red")
        result.set("\n\n\n\t\t    "+str_name)


def take_pic():
    global final_name
    global folder
    global button_pic2
    count = 0
    os.mkdir('C:\\test_face\\Face-recognition\\Faces\\' + final_name)
    path = 'C:/test_face/Face-recognition/Faces/' + final_name + '/'

    i = 0
    cap = cv2.VideoCapture(0)
    while (1):
        count = i
        ret, frame = cap.read()
        cv2.putText(frame, str(count), (12, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,
                    (0, 0, 0), 1, cv2.LINE_AA)
        k = cv2.waitKey(1)

        if k == 27:         #esc
            if count > 0:
                button_pic2['state'] = tk.NORMAL
            break
        elif k == ord('s'):
            cv2.imwrite(path + str(i) + '.jpg', frame)
            i += 1
        cv2.imshow("capture", frame)

    cap.release()
    cv2.destroyAllWindows()


def sign_up():
    global pic
    tkinter.messagebox.showinfo(title='註冊', message='註冊中，請稍後....')
    pic.destroy()
    os.system("python train.py")
    tkinter.messagebox.showinfo(title='註冊', message='註冊完成!!')


def picc():
    global final_name
    global pic
    global entry
    global result
    global button_pic
    global button_pic1
    global button_pic2
    #pic = tk.Tk()
    pic = tk.Toplevel()
    #name=tk.StringVar()
    pic.title('註冊介面')
    pic.geometry("720x550+650+200")
    
    pic.resizable(width=0, height=0)
    img_sign = ImageTk.PhotoImage(Image.open('註冊介面.png'))
    
    result = tk.StringVar()
    
    label_pic = tk.Label(pic, text='姓名', justify=tk.LEFT, image=img_sign, compound=tk.CENTER, font=('Arial', 18), textvariable=result, fg='red')
    label_pic.pack()

    # 輸入欄位
    entry = tk.Entry(pic)
    #name2 = name.get()
    entry.place(x = 285, y = 305,width=120,height=23)
    # print(1111)

    # 按鈕
    button_pic = tk.Button(pic, text="檢查", height=2, width=15, bd=5, command=hello)
    button_pic.place(x=120, y=400)

    button_pic1 = tk.Button(pic, text="拍照", height=2, width=15, bd=5, state=tk.DISABLED, command=take_pic)
    #flag1 = 1
    button_pic1.place(x=310, y=400)

    button_pic2 = tk.Button(pic, text="註冊", height=2, width=15, bd=5, state=tk.DISABLED, command=sign_up)
    button_pic2.place(x=500, y=400)
    pic.mainloop()


def des():  #初始介面結束
    global end_flag
    global user_flag
    user_flag = 2
    end_flag = 1
    window.destroy()

In [ ]:
主程式

In [ ]:
if __name__ == "__main__":
    
    required_shape = (160,160)
    face_encoder = InceptionResNetV2()
    path_m = "facenet_keras_weights.h5"
    face_encoder.load_weights(path_m)
    encodings_path = 'encodings/encodings.pkl'
    face_detector = mtcnn.MTCNN()
    encoding_dict = load_pickle(encodings_path)
    flag = 0
    #global final_name
    #global final_counter
    while(user_flag==0):
        window = tk.Tk()
        window.title('健身小幫手')
        window.geometry("720x530+650+200")
        window.resizable(width=0, height=0)
        imgg= ImageTk.PhotoImage(Image.open('初始介面.jpg'))
        # 建立按鈕
        label = tk.Label(window,text = '\n\n請辨識人臉登入', justify=tk.LEFT, image = imgg, compound = tk.CENTER, font = ('華文行楷', 30), fg = "white")  
        label.pack()
        button = tk.Button(window, text = "登入", height = 2, width = 15, bd=5, command = sign_in)
        button.place(x=120,y=400)
        button1 = tk.Button(window, text = "註冊", height = 2, width = 15, bd=5, command = picc)
        button1.place(x=310,y=400)
        #print('flag: '+str(flag))
        button2 = tk.Button(window, text = "離開", height = 2, width = 15, bd=5, command = des)
        #flag = 1
        button2.place(x=500,y=400)
        window.mainloop()